Steps:


1. Create data model
    - attributes
    - flags
    - groups

2. Process data model
    - index nodes 
    - infer nodes 
    - detect networks
    
**3. Explore networks**

4. Generate AI reports

# 3. Explore networks

## Detect networks

In [ ]:
#%pip install ucimlrepo
import sys
from ucimlrepo import fetch_ucirepo
import polars as pl

sys.path.append("../..")

from toolkit.detect_entity_networks.prepare_model import build_model_with_attributes


computer_hardware  = fetch_ucirepo(id=29)


input_dataframe = pl.DataFrame(computer_hardware.data.original)
pl.Config.set_fmt_str_lengths(200)
input_dataframe.head()

entity_id_column = "VendorName"
columns_to_link = ["ModelName", "MYCT", "MMIN", "MMAX", "CACH", "CHMIN", "CHMAX", "PRP", "ERP"]


main_graph = build_model_with_attributes(input_dataframe, entity_id_column, columns_to_link)

In [ ]:
from collections import defaultdict


max_attribute_degree = 10
max_network_size=20
supporting_attribute_types=[]
inferred_links = defaultdict(set)
additional_trimmed_attributes=[]
integrated_flags = pl.DataFrame()
group_links = []

In [ ]:
#Identify
from toolkit.detect_entity_networks.identify_networks import build_entity_records, build_networks, trim_nodeset


(trimmed_degrees, trimmed_nodes) = trim_nodeset(
    main_graph,
    additional_trimmed_attributes,
    max_attribute_degree,
)

(
    community_nodes,
    entity_to_community,
) = build_networks(
    main_graph,
    trimmed_nodes,
    inferred_links,
    supporting_attribute_types,
    max_network_size,
)

entity_records = build_entity_records(
    community_nodes,
    integrated_flags,
)

## Network data by Entity 

In [ ]:
entity_df = pl.DataFrame(
                    entity_records,
                    schema=[
                        "Entity ID",
                        "Entity Flags",
                        "Network ID",
                        "Network Entities",
                        "Network Flags",
                        "Flagged",
                        "Flags/Entity",
                        "Flagged/Unflagged",
                    ],
                ).sort("Flagged/Unflagged", descending=True)

entity_df.head()

In [ ]:
from toolkit.detect_entity_networks.explore_networks import build_network_from_entities, get_entity_graph
from toolkit.detect_entity_networks.config import ENTITY_LABEL

# Select a network
selected_network_id = entity_df.select(pl.col("Network ID").first()).item()

c_nodes = community_nodes[selected_network_id]
network_entities_graph = build_network_from_entities(
    main_graph,
    entity_to_community,
    integrated_flags,
    trimmed_degrees,
    inferred_links,
    c_nodes,
)

entity_selected=""

columns_to_link.append(ENTITY_LABEL)
nodes, edges = get_entity_graph(
    network_entities_graph,
    entity_selected,
    columns_to_link,
)

In [ ]:
pl.DataFrame(nodes).head()

In [ ]:
pl.DataFrame(edges).head()